In [1]:
from nose.tools import assert_equal

# Assignment 3 (Individual)


Continuing from Assignment 2, we will now further practice processing datasets.
From what you have seen in the lecture, in this assignment you should practice the following steps:

* basic dataset decription & assessing "tidyness"
* filtering
* sorting
* aggregation

using both "pure" Python and also using the pandas package. 

-----

## Step 1a (4 points)

Find a CSV dataset online (similar requirements apply as for Assignment 2), but additionally make sure that the dataset has 
 * at least one non-numeric column (with a categorical variable), and
 * at least one column with numerical values.

Note, as opposed to working in a group for Assignment 2, you are not allowed to use the same dataset, i.e., we expect everybody to find their own CSV file.

Save the file into your data folder and name it __data_notebook-1_DataFile.csv__.

*<b>ATTENTION</b>:* to avoid any unexpected errors when running your submission in our grading environment, we strongly recommend using an RFC-compliant CSV file encoded in `utf-8`: i.e., make sure that the CSV file uses ',' as a delimiter (if the original file doesn't, you may need to convert it first: i.e., our hidden tests assume that the code will work for any RFC-compliant CSV file with the delimiter ','.*). Lastly, please avoid CSV files with completely empty columns, i.e., each column should contain at least one value.

Using the CSV package, write a function `analyzeCSV` to return
* the number of rows in the dataset (excl. the header row).
* How many different __values__ and what __datatype__ appear per column in the CSV file.

That is, the function should return a dictionary of the following structure:
```
 {
  "rows": ..., # the number of rows (integer)
  "columns": ... # a list of pairs [ (v_1,dt_1) , (v_2,dt_2), ... (v_1,dt_1)]
 }
```
`(v_i,dt_i)` denotes the number of different values (`v_i`) and the datatype (`dt_i`) detected in column `i`.
If you detect several different datatypes per column your function should return the value `"object"` for `dt_i`. Missing value notations like `NaN` should be considered as strings. Also, you do not have to specifically test for type `datetime`.
<br><br>

Additionally, we have uploaded a CSV testfile in `unit3/data/testfile.csv` that you can use to check your solution against our target in the visible test cases. Make sure to put it in your `/data/` folder, to make the respective tests work; it also recommended to take a look at the visible test case results for all tasks, to see how you should format your output.

__Hint__: For (heuristic) datatype detection per column, please use/adapt the function `convert()` from the notebook `unit3/00_value-transformation.ipynb` from the lecture, i.e., possible return values besides the string `"object"` should be the return value of the function `type()` applied to the heuristically converted values in the column using `convert()`.

__Caution__: Do not import any other packages than those mentioned in lecture examples.

In [35]:
import pandas as pd
import csv
import os
filePath = "./data/testfile.csv"
testfile = "./data/testfile.csv"

In [98]:
filePath = "./data/data_notebook-1_DataFile.csv"

def analyzeCSV(filePath):
    row_count = 0
    values = []
    types = []
    
    with open(filePath) as f:
        reader = csv.reader(f)
        header = next(reader)  #excl. header
        column_count = len(header)  
        
        for v in range(column_count): 
            # 'set' removes duplicating values
            values.append(set())  # creating set for storing unique values from the column
            types.append(set())   # creating set for storing data types 
        
        for row in reader:
            row_count += 1
            for i in range(column_count):
                value = row[i]
                values[i].add(value)  # going through every row ain every column and storing values and types
                
                column_type = convert(value)
                types[i].add(column_type)

    columns = []
    for i in range(column_count):
        if len(types[i]) == 1:   # if there is only 1 data type, we add to the list
            columns.append((len(values[i]), types[i].pop()))
        else:
            columns.append((len(values[i]), "object")) #if there is multiple types in a single column, it will be classified as "object"
            

    result = {"rows":row_count, "columns":columns}
    return result
            

def convert(value): #convert function from the 1st unit
    try:
        int(value)
        return "int"
    except ValueError:
        pass
    try:
        float(value) 
        return "float"
    except ValueError:
        pass
    if value == "" or value.lower() == "nan":
        return "str"
    
    return "str"

#analyzeCSV(testfile)
#analyzeCSV(filePath)

In [99]:
assert_equal(type(analyzeCSV(filePath)), dict)
assert_equal(len(analyzeCSV(filePath)), 2)
assert_equal(analyzeCSV(testfile), {'rows': 12, 'columns': [(3, 'str'), (3, 'int'), (2, 'str'), (6, 'int')]})

Now (possibly with some more manual inspection) answer the following questions:
* Which types of variables appear in this dataset? For each variable, indicate whether
  *  it is numerical or categorical; for numeric values, specify whether the scale is nominal, ordinal, interval, or ratio.
  *  it is an identifier, a dimension, or a measurement.  
* How would you describe an "observation" in this dataset?
* Is the dataset tidy?
* If not, why not?

* Categorical:
     * Nominal: name, country, category 
     
* Numerical:
     * Ordinal: rownames, rank
     * Ratio: sales, profits, assets, marketvalue
     
* Identifiers:rownames, rank, name
* Dimensions: country, category
* Measurements: sales, profits, assets, marketvalue

An "observation" in this dataset is a company, characterized by ots name and net worth measures, as well as categoty and country. Each observation gives an insight on company's ranking and financial status.

Yes, the dataset is tidy:
Each row is single observation (corresponds to data of exactly 1 company).
Each value is sorted and in the right place.
Every column represents a single variable.

## Step 1b (2 points)

Now, perform the same steps you solved in 1a using pandas, i.e., read in the CSV file and write the function `pandalyzeCSV` that computes the same structure as defined in 1a.

__Hint:__ pandas dataframes provide the useful function `nunique()` to compute the number of unique values in a column (`v_i`) as well as the attribute  `dtypes` to get the column datatypes (`dt_i`), which you should use. This means, that the result returned by this solution using pandas might be slighlty different than the solution of step 1a.

Using the test CSV provided by us, your solution for `pandalyzeCSV` should result in this dictionary:

`{'rows': 12, 'columns': [(3, dtype('O')), (3, dtype('int64')), (2, dtype('O')), (6, dtype('int64'))]})`

In [39]:
def pandalyzeCSV(filePath):
    df = pd.read_csv(filePath)
    row_count = df.shape[0]
    columns = []
    
    for col in df.columns:
        values_count = df[col].nunique()
        column_type = df[col].dtype
        
        columns.append((values_count, column_type))
    
    result = {
        "rows": row_count,
        "columns": columns
    }
    
    return result

In [40]:
assert_equal(type(pandalyzeCSV(filePath)), dict)
assert_equal(len(pandalyzeCSV(filePath)), 2)

## Step 2a Filtering (2 points)

Using pure Python, i.e. NOT pandas, write the function csvFilter, which should:

* convert the CSV to a list of lists, where each row becomes an inner list;
* filter the rows of the dataset by the <b>numeric</b> column with the most unique values (if there are several columns with the same number of unique values, take the left-most of those) by showing all rows where the value of that column is less than or equal to the median of the column;
* return the list of lists containing only the filtered rows and no header!



In [104]:
#filePath = "./data/data_notebook-1_DataFile.csv"

def csvFilter(filePath):
    with open(filePath) as f:
        reader = csv.reader(f)
        data = list(reader)
        
        header = data[0]
        rows = data[1:]
        
        num_column = []
        for i in range(len(header)):
            values_column = []
            is_numeric = True
            
            for row in rows:  # identifing numeric colums
                try:
                    values_column.append(float(row[i])) # if it can be converted to the float, column found
                except ValueError:
                    is_numeric = False  
                    break # stops checking the column if non-numeric value was founded
            
            if is_numeric:  #adding fully numeric columns to set
                unique_values = set(values_column)
                num_column.append((i, len(unique_values), values_column))
        
        # Find the column with the most unique values
        max_index = 0
        max_count = num_column[0][1]
        for i in range (len(num_column)): #finding out wich column has the highest unique value count
            if num_column[i][1] > max_count:
                max_index = i
                max_count = num_column[i][1]
        
        
        selected_column = num_column[max_index]  # Get the tuple for the column with the most unique values
        max_index = selected_column[0]  # The actual column index
        max_value = selected_column[2]  #the list of numeric values in that column, in a tuple
        
        sorted_list = sorted(max_value)
        
        n = len(sorted_list)
        if n%2 != 0:
            median = sorted_list[n//2]
        else:
            median = (sorted_list[n//2-1] + sorted_list[n//2])/2
        
        
        filtered = []
        for row in rows:
            try:
                if float(row[max_index]) <= median:
                    if row not in filtered:  # Check if the row is already added
                        filtered.append(row)
            except ValueError:
                continue 
        return filtered
            
#csvFilter(testfile)
#csvFilter(filePath)

In [105]:
assert_equal(type(csvFilter(testfile)), list)
assert_equal(type(csvFilter(filePath)), list)

## Step 2b Filtering - pandas (2 points)

Now, again using pandas, write the function `pandasFilter` to:
* convert the dataset to a pandas dataframe
* filter the rows of the dataset by the same condition as for 2a:
* return the pandas dataframe only containing the filtered rows! The header and index can remain the same.

In [106]:
def pandasFilter(filePath):
    df = pd.read_csv(filePath)
    numeric_df = df.select_dtypes(include=['number'])  # Selecting only numeric columns
    unique_counts = numeric_df.nunique()  # Counting unique values per numeric column
    most_unique_column = unique_counts.idxmax()  # Finding the column with the most unique values
    median_value = numeric_df[most_unique_column].median()
    filtered_df = df[df[most_unique_column] <= median_value]  #Filtering rows where the value in the most unique column is <= median
    
    return filtered_df

In [107]:
assert_equal(type(pandasFilter(testfile)), pd.DataFrame)
assert_equal(type(pandasFilter(filePath)), pd.DataFrame)

## Step 3a Sorting (2 points)

Using pure Python, i.e. NOT using pandas:
* define a function sortCSV(filePath) which sorts the rows of your csv file by the left-most numerical column in descending order of values.
* The function takes *filePath* as input (which you have already defined above).
* Your function should return the sorted contents of the CSV file as a list of lists.


In [51]:
def sortCSV(filePath):
    with open(filePath) as f:
        reader = csv.reader(f)
        data = list(reader)

    header = data[0]
    rows = data[1:]

    numeric_index = 0   # Identifing the first numeric column
    for i in range(len(header)):
        try:
            float(rows[0][i])
            numeric_index = i    #If there is no errors, then we take the first column, that was successfully converted 
            break
        except ValueError:
            continue

    for i in range(len(rows)):    # Selection sort in descending order
        max_index = i
        for j in range(i, len(rows)):
            try:
                if float(rows[j][numeric_index]) > float(rows[max_index][numeric_index]):
                    max_index = j
            except ValueError:     # Skiping rows that cannot be converted to float
                continue
        rows[i], rows[max_index] = rows[max_index], rows[i]

    result = [header] + rows

    return result

#sortCSV(filePath)

In [52]:
assert_equal(type(sortCSV(testfile)), list)
assert_equal(type(sortCSV(filePath)), list)

## Step 3b Sorting - pandas (2 points)

Now, again using pandas, 
* define a similar function PandasSortCSV(filePath) which sorts the rows of the dataset by the same columns as in 3a, again in descending order of values. 
* This time, your function should return a pandas dataframe.
  
*Hint*: There is no need to reset the index, just leave it as it is.


In [54]:
def PandasSortCSV(filePath):
    df = pd.read_csv(filePath)
    
    numeric_column = 0
    for column in df.columns:
        if pd.to_numeric(df[column], errors='coerce').notna().all():  # If the entire column is numeric
            numeric_column = column
            break
    
    sorted_df = df.sort_values(by=numeric_column, ascending=False)
    return sorted_df


In [55]:
assert_equal(isinstance(PandasSortCSV(testfile), pd.DataFrame), True)
assert_equal(isinstance(PandasSortCSV(filePath), pd.DataFrame), True)

## Step 4a Aggregation and Grouping (4 points)

Using pure Python, i.e. NOT using pandas define a function `aggregateCSV(filePath)` which does the following:
* takes the left-most numeric column _X_ and the left-most non-numeric column _Y_ and
* calculates the __average (mean)__ of the values in column _X_ for each value in column _Y_.

Your function should return a dictionary with the unique values of the non-numeric column as keys and the averages (per group) of the numeric column as values.

In [84]:
def aggregateCSV(filePath):    
    with open(filePath) as f:
        reader = csv.reader(f)
        data = list(reader)
        
        header = data[0]
        rows = data[1:]
        
        numeric_index = None
        non_numeric_index = None
        
        for i in range (len(header)):
            try:
                float(rows[0][i])
                numeric_index = i    #If there is no errors, then we take the index of the first column, that was successfully converted 
            except ValueError:
                if non_numeric_index == None: # In contraty, the first column that wasnt successfuly converted is non_numeric
                    non_numeric_index = i
                continue
        
        values = {} # dictinary with key and all found values
        for row in rows:
            try:
                numeric_value = float(row[numeric_index])  # here we insert the index and find the according value
                non_numeric_value = str(row[non_numeric_index])
                
                if non_numeric_value not in values:   #if there is no key with Non_numeric_value
                    values[non_numeric_value] = []    # it creates such key, with an empty list as a value
                values[non_numeric_value].append(numeric_value) # and now we can add all numeric values that have the same non_numeric key
            except ValueError:
                continue
        
        result = {}  # dictinary with key and mean of all found values
        for key, values in values.items():
            mean = sum(values)/len(values)
            result[key] = mean
                
        return result

#aggregateCSV(testfile) 
#aggregateCSV(filePath)
                 

In [85]:
assert_equal(type(aggregateCSV(testfile)), dict)
assert_equal(type(aggregateCSV(filePath)), dict)

## Step 4b Aggregation and Grouping - pandas (2 points)

Now, again using pandas, do the same as in 4a and call the function `PandasAggregateCSV` this time. It should return a pandas dataframe. The columns do not have to be renamed.


In [95]:
def PandasAggregateCSV(filePath):
    df = pd.read_csv(filePath)
    
    numeric_column = None
    non_numeric_column = None
    
    for column in df.columns:
        try:                            #converting to float to check if the column numeric
            float(df[column].iloc[0])   # [0] it is accessing the first row of the specified column
            if numeric_column is None:
                numeric_column = column
        except ValueError:              # If it fails, it is non-numeric
            if non_numeric_column is None:
                non_numeric_column = column
                
        if (numeric_column != None) and (non_numeric_column != None):
            break                       # break when columns are found
            
    result = df.groupby(non_numeric_column)[numeric_column].mean().reset_index()   #Tutorium 3
    
    return result

In [93]:
assert_equal(isinstance(PandasAggregateCSV(testfile), pd.DataFrame), True)
assert_equal(isinstance(PandasAggregateCSV(filePath), pd.DataFrame), True)